In [6]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, List
from IPython.display import Image, display 
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from pydantic import BaseModel, Field

In [5]:
essay = """The rapid growth of technology has transformed nearly every aspect of human life. From communication to transportation, innovation continues to push boundaries and redefine possibilities. Artificial intelligence, for instance, has not only automated repetitive tasks but also enhanced creativity and decision-making across industries. However, this progress also brings challenges, such as ethical dilemmas, privacy concerns, and the risk of overdependence on machines. To truly benefit from technological advancement, society must strike a balance between innovation and responsibility. Education, awareness, and thoughtful regulation can ensure that technology remains a tool for empowerment rather than a force of disruption."""

In [ ]:
"""
Trying to get structured output

# 1. Defining the output schema
class output_schema(BaseModel):
    feedback : str = Field(description="This stores the feedback from the llm")
    score : int = Field(description="This stores the score of the essay")

# 2. Initializing the model
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash").with_structured_output(output_schema)

# 3. Creating the Prompt
prompt = PromptTemplate.from_template("Provide a detailed feedback and a score to the following essay from 0-10\n\n essay = {essay}")

# 4. Invoking the model
response = llm.invoke(prompt.format(essay=essay))

print(response.feedback)
print(response.score)

"""

The essay provides a concise yet comprehensive overview of the impact of technology. It effectively highlights both the transformative benefits, such as AI's role in automation and enhanced decision-making, and the critical challenges, including ethical dilemmas and privacy concerns. The structure is clear, moving logically from impact to challenges and then offering thoughtful solutions like education and regulation. The language used is precise and professional. To further strengthen the argument, you could consider providing more specific examples for each point, such as concrete instances of AI enhancing creativity or particular ethical dilemmas that arise. Overall, it's a well-articulated piece that demonstrates a strong understanding of the topic.
8


In [13]:
class UPSCstate(TypedDict):
    essay_text : str
    cot_feedback : str
    doa_feedback : str
    lang_feedback : str
    scores : List[int]
    final_feedback : str
    final_score : int


In [ ]:
graph = StateGraph(UPSCstate)
